In [0]:
import pandas as pd 
import numpy as np 
import math
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.tools as tls
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import os
import pickle

In [0]:
pos_embedding = pickle.load(open('pos_embedding_vec_without_one_hot_encoding','rb'))
len(pos_embedding)

6291

In [0]:

pos_embedding[1].shape
type(pos_embedding[0])

numpy.ndarray

In [0]:
neg_embedding = pickle.load(open('neg_embedding_vec_without_one_hot_encoding','rb'))

In [0]:
len(neg_embedding)

187232

In [0]:
#sub smaple negative samples
neg_embedding=neg_embedding[:6291]
len(neg_embedding)



6291

In [0]:
#concatenating
all_embeddings=list(pos_embedding+neg_embedding)
len(all_embeddings)

12582

In [0]:
#creating two arrays with 1s and 0s
pos_target=np.ones(6291)
pos_target



array([1., 1., 1., ..., 1., 1., 1.])

In [0]:
neg_target=np.zeros(6291)
neg_target

array([0., 0., 0., ..., 0., 0., 0.])

In [0]:
all_target= np.append(pos_target,neg_target,axis=None)
all_target

array([1., 1., 1., ..., 0., 0., 0.])

In [0]:
all_target[:5]

array([1., 1., 1., 1., 1.])

In [0]:
type(all_embeddings)

list

In [0]:
all_embeddings[0].shape

(361, 6)

In [0]:
embedding_size = 361
question_size = 104
def get_padded_embeddings(temp_df):
    #print("tdf", temp_df.shape)
    temp_df=temp_df.reshape(-1,361)
    zero_embeddings = np.zeros(embedding_size)
    truncated_df = temp_df[:question_size]
    #print(truncated_df.shape)
    #print(len(truncated_df),len(truncated_df[0]))
    #print(type([zero_embeddings]*(question_size-len(truncated_df))))
    #print(np.array([zero_embeddings]*(question_size - len(truncated_df))).shape)
    if(len(truncated_df)!=104):
        truncated_df = np.concatenate((truncated_df, np.array([zero_embeddings]*(question_size- len(truncated_df)))))
        #print(truncated_df.shape)
    return truncated_df


In [0]:
emb_vect_temp=np.array([get_padded_embeddings(line) for line in all_embeddings])

In [0]:
emb_vect_temp.shape

(12582, 104, 361)

In [0]:
type(emb_vect_temp)

numpy.ndarray

In [0]:
all_embeddings_temp, all_embeddings_test,y_temp,y_test = train_test_split(emb_vect_temp, all_target, test_size = 0.30, shuffle=True, random_state = 42)


In [ ]:
all_embeddings_train,all_embeddings_val,y_train,y_val= train_test_split(all_embeddings_temp,y_temp,test_size=0.20)


In [0]:
for i in range(10):
    print(all_embeddings_train[i].shape)

(361, 2)
(361, 10)
(361, 12)
(361, 12)
(361, 2)
(361, 10)
(361, 7)
(361, 11)
(361, 1)
(361, 6)


In [0]:
a=all_embeddings_train[2].reshape(-1,361).shape
a

(12, 361)

In [0]:
a=get_padded_embeddings(all_embeddings_train[2])

tdf (361, 12)
(12, 361)
12 361
<class 'list'>
(92, 361)
(104, 361)


In [0]:
#all_embeddings_train[1].reshape(-1,361)
#all_embeddings_train[1].shape

In [0]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional,Flatten
from sklearn.metrics import f1_score
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True, input_shape=(104,361))))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(1, activation="sigmoid"))

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
history=model.fit(all_embeddings_train, y_train, validation_data=(all_embeddings_val,y_val),batch_size=None,nb_epoch=20,steps_per_epoch=1000, verbose=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

The `nb_epoch` argument in `fit` has been renamed `epochs`.



Epoch 1/20
  55/1000 [>.............................] - ETA: 15:11:50 - loss: 0.6105 - acc: 0.6590

In [ ]:
with open('History', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

with open('Model', 'wb') as file_mo:
        pickle.dump(model, file_mo)



In [ ]:
score=model.evaluate(all_embeddings_test,y_test,steps_per_epoch=1000,verbose=1)
print("Test Accuracy",score[1])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 104, 64)           109056    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 142,145
Trainable params: 142,145
Non-trainable params: 0
_________________________________________________________________


In [0]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()